# Codigo Final - EPP / Deteccion / Reconocimiento 

In [1]:
import tkinter as tk
from tkinter import Label
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
import random
from ultralytics import YOLO
import face_recognition as fr
import os
from datetime import datetime
from openpyxl import Workbook, load_workbook

# Variables globales
cap = None
img_original = None
deteccion_activa = False
reconocimiento_activo = False
model = YOLO('best6.pt')

path = 'Personal'
images = []
nombres = []
lista = os.listdir(path)

# Flag global para saber si la cámara está activa
camera_active = False

# Cargar imágenes de EPP
def cargar_imagenes():
    base_path = os.path.abspath("Images")
    global img_no_casco, img_si_casco, img_no_gafas, img_si_gafas, img_no_chaleco, img_si_chaleco
    global img_no_persona, img_si_persona, img_no_guantes, img_si_guantes, img_no_safe, img_si_safe

    img_no_casco = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "no_casco.png")))
    img_si_casco = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "si_casco.png")))
    img_no_gafas = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "no_gafas.png")))
    img_si_gafas = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "si_gafas.png")))
    img_no_chaleco = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "no_chaleco.png")))
    img_si_chaleco = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "si_chaleco.png")))
    img_no_persona = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "no_persona.png")))
    img_si_persona = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "si_persona.png")))
    img_no_guantes = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "no_guantes.png")))
    img_si_guantes = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "si_guantes.png")))
    img_no_safe = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "no_safe.png")))
    img_si_safe = ImageTk.PhotoImage(Image.open(os.path.join(base_path, "si_safe.png")))

# Actualizar etiquetas de EPP
def update_epp_status(casco_detected, gafas_detected, chaleco_detected, persona_detected, guantes_detected, safe_detected):

    if casco_detected:
        lbl_casco.config(image=img_si_casco)
    else:
        lbl_casco.config(image=img_no_casco)

    if gafas_detected:
        lbl_gafas.config(image=img_si_gafas)
    else:
        lbl_gafas.config(image=img_no_gafas)

    if chaleco_detected:
        lbl_chaleco.config(image=img_si_chaleco)
    else:
        lbl_chaleco.config(image=img_no_chaleco)

    if persona_detected:
        lbl_persona.config(image=img_si_persona)
    else:
        lbl_persona.config(image=img_no_persona)

    if guantes_detected:
        lbl_guantes.config(image=img_si_guantes)
    else:
        lbl_guantes.config(image=img_no_guantes)

    if safe_detected:
        lbl_safe.config(image=img_si_safe)
    else:
        lbl_safe.config(image=img_no_safe)

for archivo in lista:
    imgdb = cv2.imread(f'{path}/{archivo}')
    images.append(imgdb)
    nombres.append(os.path.splitext(archivo)[0])

def codificar_rostros(images):
    lista_codificaciones = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        codificacion = fr.face_encodings(img)[0]
        lista_codificaciones.append(codificacion)
    return lista_codificaciones

rostros_codificados = codificar_rostros(images)

def horario(nombre):
    archivo_excel = "Horario.xlsx"
    if not os.path.exists(archivo_excel):
        wb = Workbook()
        ws = wb.active
        ws.append(["Nombre", "Fecha", "Hora"])
        wb.save(archivo_excel)
    wb = load_workbook(archivo_excel)
    ws = wb.active
    fecha_actual = datetime.now().strftime('%Y-%m-%d')
    for fila in ws.iter_rows(min_row=2, values_only=True):
        if fila[0] == nombre and fila[1] == fecha_actual:
            return
    hora_actual = datetime.now().strftime('%H:%M:%S')
    ws.append([nombre, fecha_actual, hora_actual])
    wb.save(archivo_excel)

def realizar_reconocimiento(frame):
    global rostros_codificados, nombres
    frame_pequeño = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    frame_pequeño = cv2.cvtColor(frame_pequeño, cv2.COLOR_BGR2RGB)
    ubicaciones_rostros = fr.face_locations(frame_pequeño, model='hog')
    codificaciones_rostros = fr.face_encodings(frame_pequeño, ubicaciones_rostros, num_jitters=1)

    for codificacion_rostro, ubicacion_rostro in zip(codificaciones_rostros, ubicaciones_rostros):
        coincidencias = fr.compare_faces(rostros_codificados, codificacion_rostro, tolerance=0.6)
        nombre = "Desconocido"
        if True in coincidencias:
            indice_coincidencia = coincidencias.index(True)
            nombre = nombres[indice_coincidencia]
            horario(nombre)

        top, right, bottom, left = ubicacion_rostro
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, nombre, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    return frame

class_mapping = {
    'casco': 'Casco',
    'sin casco': 'Sin Casco',
    'chaleco': 'Chaleco',
    'sin chaleco': 'Sin Chaleco',
    'gafas': 'Gafas',
    'sin gafas': 'Sin Gafas',
    'guante': 'Guantes',
    'persona': 'Persona'
}

color_cache = {}

def get_color_for_class(class_name):
    if class_name not in color_cache:
        color_cache[class_name] = tuple(random.randint(0, 255) for _ in range(3))
    return color_cache[class_name]

def reiniciar_estado():
    global cap, deteccion_activa, reconocimiento_activo
    if cap is not None:
        cap.release()
        cap = None
    deteccion_activa = False
    reconocimiento_activo = False
    print("Estado reiniciado.")


def cargar_archivo():
    global cap, img_original
    archivo = filedialog.askopenfilename(
        title="Seleccionar archivo", 
        filetypes=[("Archivos de imagen o video", "*.jpg;*.jpeg;*.png;*.mp4;*.avi")]
    )
    if archivo:
        if archivo.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_original = cv2.imread(archivo)
            img_original = cv2.resize(img_original, (600, 400))
            img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
            mostrar_imagen(img_original)
        elif archivo.lower().endswith(('.mp4', '.avi')):
            cap = cv2.VideoCapture(archivo)
            mostrar_video()


def activar_camara():
    global cap
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print("No se pudo acceder a la cámara.")
        return
    camera_active = True
    mostrar_video()
    

def mostrar_video():
    casco_detected = False
    gafas_detected = False
    chaleco_detected = False
    persona_detected = False
    guantes_detected = False
    safe_detected = False

    global cap
    if cap is not None and cap.isOpened():
        ret, frame = cap.read()
        if ret:
            print("Frame leído correctamente.")  # Depuración
            frame_copy = frame.copy()
            
            # Reconocimiento facial
            if reconocimiento_activo:
                frame_copy = realizar_reconocimiento(frame_copy)
            
            # Detección de EPP
            if deteccion_activa:
                try:
                    results = model(frame_copy, conf=0.25)
                    frame_copy = procesar_video_frame(frame_copy)
                    
                    # Extraer estado de detección
                    casco_detected = is_class_detected(results, 0) 
                    gafas_detected = is_class_detected(results, 2)  
                    chaleco_detected = is_class_detected(results, 1)  
                    persona_detected = is_class_detected(results, 4)  
                    guantes_detected = is_class_detected(results, 3)  
                    safe_detected = casco_detected and gafas_detected and chaleco_detected

                    # Actualizar las etiquetas
                    update_epp_status(casco_detected, gafas_detected, chaleco_detected, persona_detected, guantes_detected, safe_detected)
                except Exception as e:
                    print(f"Error en detección de EPP: {e}")

            # Mostrar el frame procesado
            frame_to_show = cv2.cvtColor(frame_copy, cv2.COLOR_BGR2RGB)
            frame_resized = escalar_y_centrar(frame_to_show, (600, 400))
            img_tk = ImageTk.PhotoImage(image=Image.fromarray(frame_resized))
            lblVideo.configure(image=img_tk, text="")
            lblVideo.image = img_tk
            lblVideo.after(20, mostrar_video)  # Llama al siguiente frame
        else:
            print("No se pudo leer el frame. Fin del video o error.")
            cap.release()
            cap = None
    else:
        print("Error: Cámara o archivo de video no está disponible.")


def procesar_imagen(img):
    results = model(img, conf=0.25)
    return dibujar_cajas(img.copy(), results)

def procesar_video_frame(frame):
    results = model(frame, conf=0.25)
    return dibujar_cajas(frame.copy(), results)

def dibujar_cajas(img, results):
    for result in results:
        if hasattr(result, 'boxes'):
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                class_id = int(box.cls[0].item())
                confidence = box.conf[0].item()
                class_name = model.names[class_id] if class_id in range(len(model.names)) else "Desconocido"
                label = class_mapping.get(class_name, "Desconocido")
                color = get_color_for_class(class_name)
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, f"{label} ({confidence:.2f})", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return img

def is_class_detected(results, class_id):
    """
    Verifica si una clase específica fue detectada en los resultados del modelo.
    """
    for result in results:
        if hasattr(result, 'boxes') and result.boxes:
            for box in result.boxes:
                if int(box.cls[0].item()) == class_id:
                    return True
    return False


def mostrar_imagen(img):
    img_tk = ImageTk.PhotoImage(image=Image.fromarray(img))
    lblVideo.configure(image=img_tk, text="")
    lblVideo.image = img_tk

def escalar_y_centrar(img, target_size):
    target_width, target_height = target_size
    h, w, _ = img.shape
    scale = min(target_width / w, target_height / h)
    new_width = int(w * scale)
    new_height = int(h * scale)
    img_resized = cv2.resize(img, (new_width, new_height))
    canvas = np.zeros((target_height, target_width, 3), dtype=np.uint8)
    x_offset = (target_width - new_width) // 2
    y_offset = (target_height - new_height) // 2
    canvas[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = img_resized
    return canvas

def abrir_excel_directamente():
    archivo_excel = "Horario.xlsx"  # Ruta del archivo Excel a abrir
    if os.path.exists(archivo_excel):
        os.startfile(archivo_excel)  # Abre el archivo con la aplicación predeterminada (Excel)
    else:
        print(f"El archivo '{archivo_excel}' no existe.")

def guardar_captura():
    global img_original, deteccion_activa, reconocimiento_activo, cap

    # Caso 1: Si se cargó una imagen
    if img_original is not None:
        frame = img_original.copy()  # Copiar la imagen original

        # Aplicar detecciones si están activas
        if deteccion_activa:
            frame = procesar_imagen(frame)  # Aplicar detección de EPP
        if reconocimiento_activo:
            frame = realizar_reconocimiento(frame)  # Aplicar reconocimiento facial

        # Convertir la imagen procesada de RGB a BGR antes de guardarla
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        guardar_frame(frame_bgr)

    # Caso 2: Si se está usando la cámara o el video
    elif cap is not None and cap.isOpened():
        ret, frame = cap.read()  # Leer el frame actual del video/cámara
        if ret:
            # Aplicar detecciones si están activas
            if deteccion_activa:
                frame = procesar_video_frame(frame)  # Aplicar detección de EPP
            if reconocimiento_activo:
                frame = realizar_reconocimiento(frame)  # Aplicar reconocimiento facial
            
            # Guardar el frame procesado directamente (ya está en BGR)
            guardar_frame(frame)
        else:
            print("No se pudo capturar el frame actual.")
    else:
        print("No hay contenido activo para capturar.")

def guardar_lblVideo():
    if lblVideo.image is not None:
        # Obtener la imagen actualmente mostrada en lblVideo
        img_tk = lblVideo.image
        img_pil = ImageTk.getimage(img_tk)

        # Convertir la imagen a formato compatible con OpenCV
        img_np = np.array(img_pil)
        img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)  # Convertir de RGB a BGR

        # Guardar el frame como captura
        guardar_frame(img_np)
    else:
        print("No hay contenido en el área de visualización para guardar.")

def guardar_frame(frame):
    # Asegúrate de que exista la carpeta Resultados
    if not os.path.exists("Resultados"):
        os.makedirs("Resultados")

    # Generar un nombre único basado en la fecha y hora
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"Resultados/Captura_{timestamp}.jpg"

    # Guardar el frame procesado en formato JPEG (BGR)
    cv2.imwrite(filename, frame)
    print(f"Captura guardada: {filename}")


# Crear ventana principal
root = tk.Tk()
root.title("Detección de Equipo de Protección Personal")
root.geometry("1280x720")
root.configure(bg="#b2bec4")


# Marco superior (Título)
frame_top = tk.Frame(root, bg="#3b82f6", height=50)
frame_top.pack(fill="x")
title_label = tk.Label(frame_top, text="Detección de Equipo de Protección Personal y Reconocimiento Facial", bg="#3b82f6", fg="white", font=("Arial", 16, "bold"))
title_label.pack(pady=10)

# Panel izquierdo para estados de EPP
cargar_imagenes()  

# Imágenes ya existentes
lbl_casco = Label(root, image=img_no_casco, bg="#f0f0f0")
lbl_casco.place(x=75, y=125)
lbl_gafas = Label(root, image=img_no_gafas, bg="#f0f0f0")
lbl_gafas.place(x=75, y=285)
lbl_chaleco = Label(root, image=img_no_chaleco, bg="#f0f0f0")
lbl_chaleco.place(x=75, y=450)

# Nuevas imágenes
lbl_persona = Label(root, image=img_no_persona, bg="#f0f0f0")
lbl_persona.place(x=995, y=125)
lbl_guantes = Label(root, image=img_no_guantes, bg="#f0f0f0")
lbl_guantes.place(x=995, y=285)
lbl_safe = Label(root, image=img_no_safe, bg="#f0f0f0")
lbl_safe.place(x=995, y=450)


# Marco inferior para los botones
frame_bottom = tk.Frame(root, bg="#f0f0f0", height=100)  # Fondo gris
frame_bottom.pack(side="bottom", fill="x")

# Crear Canvas en la parte inferior
canvas = tk.Canvas(frame_bottom, bg="#f0f0f0", height=100, width=800, highlightthickness=0)
canvas.pack()

# Calcular posición inicial para centrar los botones
icon_paths = ["cargar.png", "color1.png", "gris.png", "deteccion.png", "descarga.png", "camara.png"]
canvas_width = 800
button_spacing = 80
total_buttons_width = len(icon_paths) * button_spacing
start_x = (canvas_width - total_buttons_width) // 2

images = []

def on_button_click(circle, label):
    global deteccion_activa, reconocimiento_activo
    if label == "Botón 1":
        reiniciar_estado()
        cargar_archivo()
        
    elif label == "Botón 2":
        abrir_excel_directamente()

    elif label == "Botón 3":
        reconocimiento_activo = not reconocimiento_activo
        print(f"Reconocimiento {'activado' if reconocimiento_activo else 'desactivado'}.")
    
    elif label == "Botón 4":
        deteccion_activa = not deteccion_activa
        print(f"Detección {'activada' if deteccion_activa else 'desactivada'}.")
        if img_original is not None:
            mostrar_imagen(img_original if not deteccion_activa else procesar_imagen(img_original))
    elif label == "Botón 5":
        guardar_captura()

    elif label == "Botón 6":
        reiniciar_estado()
        activar_camara()

# Cargar iconos de botones
x_position = start_x
for path in icon_paths:
    img = Image.open(path).resize((40, 40), Image.LANCZOS)
    img_tk = ImageTk.PhotoImage(img)
    images.append(img_tk)
    circle = canvas.create_oval(x_position, 20, x_position + 60, 80, fill="#FFEB3B", outline="#FFC107")
    img_id = canvas.create_image(x_position + 30, 50, image=img_tk)
    label = f"Botón {len(images)}"
    canvas.tag_bind(circle, "<Button-1>", lambda event, circle=circle, label=label: on_button_click(circle, label))
    canvas.tag_bind(img_id, "<Button-1>", lambda event, circle=circle, label=label: on_button_click(circle, label))
    x_position += button_spacing
    
# Área de visualización de video
lblVideo = tk.Label(root, text="Área de visualización de imágenes", bg="#ffffff", font=("Arial", 14))
lblVideo.place(x=320, y=120, width=640, height=480)

root.mainloop()

Estado reiniciado.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leído correctamente.
Frame leí